In [14]:
## some options
inputBibFileName = "../../_bibliography/papers.bib"
outputJSONFileName = "collab_net.json"
#
authorInformationFile = (
    ""  #'authorinfoRosalindFranklin.csv' # optional co-author information
)
deleteEgoNode = False
##

import csv  # for loading comma seperated values
import json  # for writing the json
import re

# import necessary libraries
from pybtex.database.input import bibtex  # for reading the bib files


# some auxiliary functions
# you migth have to add further repalcement rules
def latex2unicode(latexString):
    """takes the name of an author as string and return the string
    with latex character replaced as normal string for the HTML"""
    latexString = latexString.replace('{\\"u}', "ü")
    latexString = latexString.replace("{\\'o}", "ó")
    latexString = latexString.replace("{\\'a}", "á")
    latexString = latexString.replace("{\\~a}", "ã")

    return latexString


# some preperation to read the bibtex file
parser = bibtex.Parser()
bib_data = parser.parse_file(inputBibFileName)


listOfAuthors = []  # empty list of authors

In [5]:
# go throuh all entries
for paperKeys in bib_data.entries.keys():
    # get the authors of this paper
    authors = bib_data.entries[paperKeys].fields['author'].split(" and ")
    # save them to the list of authors
    for author in authors:
        listOfAuthors.append(author)


KeyError: 'author'

In [9]:
bib_data.entries[paperKeys]

Entry('unpublished',
  fields=[
    ('title', 'Social learning across symbolic cultural barriers in non-human cultures'), 
    ('journal', 'arXiv:2307.05304'), 
    ('year', '2023'), 
    ('abstract', 'Social learning is key in the development of both human and non-human animal societies. Here, we provide quantitative evidence that supports the existence of social learning in sperm whales across socio-cultural barriers, based on acoustic data from locations in the Pacific and Atlantic Oceans. Sperm whale populations have traditionally been partitioned into clans based on their vocal repertoire (what they say) of rhythmically patterned clicks (codas), and in particular their identity codas, which serve as symbolic markers for each clan. However, identity codas account for between 35% and 60% of all codas vocalized depending on the different clans. We introduce a computational modeling approach that recovers clan structure and shows new evidence of social learning across clans from the i

In [8]:
paperKeys

'leitao2023social'

In [7]:
bib_data.entries.keys()

KeysView(OrderedCaseInsensitiveDict([('leitao2023social', Entry('unpublished',
  fields=[
    ('title', 'Social learning across symbolic cultural barriers in non-human cultures'), 
    ('journal', 'arXiv:2307.05304'), 
    ('year', '2023'), 
    ('abstract', 'Social learning is key in the development of both human and non-human animal societies. Here, we provide quantitative evidence that supports the existence of social learning in sperm whales across socio-cultural barriers, based on acoustic data from locations in the Pacific and Atlantic Oceans. Sperm whale populations have traditionally been partitioned into clans based on their vocal repertoire (what they say) of rhythmically patterned clicks (codas), and in particular their identity codas, which serve as symbolic markers for each clan. However, identity codas account for between 35% and 60% of all codas vocalized depending on the different clans. We introduce a computational modeling approach that recovers clan structure and sho

In [4]:
bib_data

BibliographyData(
  entries=OrderedCaseInsensitiveDict([
    ('zhang2022do', Entry('unpublished',
      fields=[
        ('title', 'Do higher-order interactions promote synchronization?'), 
        ('year', '2022'), 
        ('arxiv', '2203.03060'), 
        ('code', 'https://github.com/maximelucas/higherorder_sync_promoted'), 
        ('bibtex_show', 'true'), 
        ('abstract', 'Higher-order interactions, through which three or more entities interact simultaneously, are important to the faithful modeling of many real-world complex systems. Recent efforts have focused on elucidating the effects of these nonpairwise interactions on the collective behaviors of coupled systems. Interestingly, several examples of higher-order interactions promoting synchronization have been found, raising speculations that this might be a general phenomenon. Here, we demonstrate that even for simple systems such as Kuramoto oscillators, the effects of higher-order interactions are highly nuanced. In par

In [17]:
for author in bib_data.entries[paperKeys].persons['author']:
    print(author)

Duncan, R.
Lucas, M.


In [18]:
author

Person('Lucas, M.')

In [19]:
bib_data.entries[paperKeys].persons['author']

[Person('Duncan, R.'), Person('Lucas, M.')]

In [20]:
# go throuh all entries
for paperKeys in bib_data.entries.keys():
    # get the authors of this paper

    listOfAuthors.append(bib_data.entries[paperKeys].persons['author'])
    #authors = bib_data.entries[paperKeys].persons['author'].split(" and ")
    # save them to the list of authors
    # for author in authors:

In [21]:
listOfAuthors=[] # empty list of authors

# go throuh all entries
for paperKeys in bib_data.entries.keys():
    # get the authors of this paper

    #authors = bib_data.entries[paperKeys].persons['author'].split(" and ")
    # save them to the list of authors
    for author in bib_data.entries[paperKeys].persons['author']:
        listOfAuthors.append(str(author))

In [22]:
listOfAuthors

['Leit{\\~a}o, A.',
 'Lucas, M.',
 'Poetto, S.',
 'Hersh, T. A.',
 'Gero, S.',
 'Gruber, D.',
 'Bronstein, M.',
 'Petri, G.',
 'Nurisso, M.',
 'Arnaudon, A.',
 'Lucas, M.',
 'Peach, R. L.',
 'Expert, P.',
 'Vaccarino, F.',
 'Petri, G.',
 'Landry, N. W.',
 'Lucas, M.',
 'Iacopini, I.',
 'Petri, G.',
 'Schwarze, A.',
 'Patania, A.',
 'Torres, L.',
 'Zhang*, Y.',
 'Lucas*, M.',
 'Battiston, F.',
 'Lucas*, M.',
 'Iacopini*, I.',
 'Robiglio, T.',
 'Barrat, A.',
 'Petri, G.',
 'Lucas, M.',
 'Morris, A.',
 'Townsend-Teague, A.',
 'Tichit, L.',
 'Habermann, B. H.',
 'Barrat, A.',
 'Lucas, M.',
 'Cencetti, G.',
 'Battiston, F.',
 'Newman, J.',
 'Lucas, M.',
 'Stefanovska, A.',
 'Lucas, M.',
 'Newman, J.',
 'Stefanovska, A.',
 'Newman, J.',
 'Lucas, M.',
 'Stefanovska, A.',
 'Lucas, M.',
 'Cencetti, G.',
 'Battiston, F.',
 'Battiston, F.',
 'Cencetti, G.',
 'Iacopini, I.',
 'Latora, V.',
 'Lucas, M.',
 'Patania, A.',
 'Young, J.-G.',
 'Petri, G.',
 'Lucas, M.',
 'Lucas, M.',
 'Fanelli, D.',
 'St

In [23]:
[author.split('*')[0].strip() for author in listOfAuthors]

['Leit{\\~a}o, A.',
 'Lucas, M.',
 'Poetto, S.',
 'Hersh, T. A.',
 'Gero, S.',
 'Gruber, D.',
 'Bronstein, M.',
 'Petri, G.',
 'Nurisso, M.',
 'Arnaudon, A.',
 'Lucas, M.',
 'Peach, R. L.',
 'Expert, P.',
 'Vaccarino, F.',
 'Petri, G.',
 'Landry, N. W.',
 'Lucas, M.',
 'Iacopini, I.',
 'Petri, G.',
 'Schwarze, A.',
 'Patania, A.',
 'Torres, L.',
 'Zhang',
 'Lucas',
 'Battiston, F.',
 'Lucas',
 'Iacopini',
 'Robiglio, T.',
 'Barrat, A.',
 'Petri, G.',
 'Lucas, M.',
 'Morris, A.',
 'Townsend-Teague, A.',
 'Tichit, L.',
 'Habermann, B. H.',
 'Barrat, A.',
 'Lucas, M.',
 'Cencetti, G.',
 'Battiston, F.',
 'Newman, J.',
 'Lucas, M.',
 'Stefanovska, A.',
 'Lucas, M.',
 'Newman, J.',
 'Stefanovska, A.',
 'Newman, J.',
 'Lucas, M.',
 'Stefanovska, A.',
 'Lucas, M.',
 'Cencetti, G.',
 'Battiston, F.',
 'Battiston, F.',
 'Cencetti, G.',
 'Iacopini, I.',
 'Latora, V.',
 'Lucas, M.',
 'Patania, A.',
 'Young, J.-G.',
 'Petri, G.',
 'Lucas, M.',
 'Lucas, M.',
 'Fanelli, D.',
 'Stefanovska, A.',
 'Lu

In [24]:

if deleteEgoNode == True:
    # we assume that the author with the most entries is the ego node and delete it
     egoNode = max(listOfAuthors, key=listOfAuthors.count) # returns the ego node
     listOfAuthors = list(set(listOfAuthors)) # gets unique list of authors
     listOfAuthors.pop(listOfAuthors.index(egoNode)) # deletes it from the list of authors
     print("Removing the ego node: %s " %egoNode )
else:
    listOfAuthors = list(set(listOfAuthors)) # gets unique list of authors

nAuthors = len(listOfAuthors) # number of author nodes
nPapers = len(bib_data.entries.keys()) # number of paper nodes


# read the additional author information from the csv
authorLinks_dict = {} # create an empty dictionary
authorImage_dict = {} # create an empty dictionary

try:
    authorInfo_reader = csv.DictReader(open(authorInformationFile))
    for row in authorInfo_reader:
        authorLinks_dict[row['name']] = row['url']
        authorImage_dict[row['name']] = row['image']
except FileNotFoundError:
    print("no optional co-author information available")


no optional co-author information available


In [25]:
str(listOfAuthors[0])

'Peach, R. L.'

In [26]:
# create a dictionary reflecting the graph (there are more pythonic ways
# possible to creat this, e.g., with zip, but this is easiest)
node_list = []
# create author nodes
for i in range(nAuthors):
    node_dict = {} # create an empty dictionary for this node
    node_dict["id"] = "A" + str(i)
    node_dict["group"] = 0
    # invert the name such that the given name is before the last name
    try: # we need this try to deal with single author papers
        authorSplit = listOfAuthors[i].split(",")
        nameThisAuthor = authorSplit[1][1::] + ' ' + authorSplit[0]
    except:
        authorSplit = listOfAuthors[i]
        nameThisAuthor = authorSplit[1][1::] + ' ' + authorSplit[0]

    nameThisAuthorUnicode = latex2unicode(nameThisAuthor)
    print(nameThisAuthorUnicode)
    node_dict["name"] = nameThisAuthorUnicode
    node_list.append(node_dict)
    # try to set the url for this author but default is google it
    node_dict["url"] = "https://www.google.com/search?q=" + nameThisAuthor
    try:
        if authorLinks_dict[nameThisAuthor] is None:
            raise KeyError('no information for this author')
        else:
            node_dict["url"] = authorLinks_dict[nameThisAuthor]
    except KeyError:
        node_dict["url"] = "https://www.google.com/search?q=" + nameThisAuthor

        # try to set a image for this author
    try:
        node_dict["image"] = authorImage_dict[nameThisAuthor]
    except KeyError: # if no image jsut leave blank
        node_dict["image"] = []

R. L. Peach
N. W. Landry
G. Cencetti
D. Gruber
T. Carletti
T. Robiglio
L. Tichit
P. Expert
A. Schwarze
J. Petit
A. Barrat
F. Vaccarino
S. Poetto
M. Lucas
A. Morris
I. Iacopini*
F. Battiston
M. Lucas*
L. Torres
J. Newman
A. Leitão
I. Iacopini
B. H. Habermann
J.-G. Young
R. Duncan
T. A. Hersh
M. Nurisso
A. Patania
G. Petri
A. Arnaudon
Y. Zhang*
A. Stefanovska
A. Townsend-Teague
M. Bronstein
V. Latora
S. Gero
D. Fanelli


In [27]:
# create the links between the nodes
link_list = []
i=0
for paperKeys in bib_data.entries.keys(): # go over every paper
    # create the paper node
    node_dict = {} # create an empty dictionary for this node
    node_dict["id"] = "P" + str(i)
    node_dict["group"] = 1
    thisPaperName = bib_data.entries[paperKeys].fields['title']
    # remove curly bracket in paper name, remove it
    if thisPaperName[0]=='{':
        thisPaperName=thisPaperName[1:-1]
    node_dict["name"] =  thisPaperName
    node_list.append(node_dict)

    # set image for this paper
    try:
        node_dict["image"] = bib_data.entries[paperKeys].fields['image']
    except KeyError: # if no image just leave blank
        node_dict["image"] = []


    # find the authors for this paper
    authorsThisPaper = [str(i).split('*')[0].strip() for i in bib_data.entries[paperKeys].persons['author']]

    # if the paper has a url, add it
    try:
        node_dict["url"] = bib_data.entries[paperKeys].fields['url']
    except KeyError: # otherwise refer to google
        node_dict["url"] = "https://www.google.com/search?q=" + bib_data.entries[paperKeys].fields['title']


    for authors in authorsThisPaper:
        link_dict = {} # empty dictionary for this edge
        link_dict["source"] = "P" + str(i) # attached to this paper
        try:
            link_dict["target"] = "A" + str(listOfAuthors.index(authors)) # and attached to co-author
            link_list.append(link_dict)    # save it into the list
        except ValueError:
            pass
            #print("Author %s not in list, probably the ego node." %authors )
    i=i+1

# write into dictionary
graph_dict = {"nodes" : node_list, "links" : link_list}


# opening the file to write
if outputJSONFileName:
    # Writing JSON data
    with open(outputJSONFileName, 'w') as f:
        json.dump(graph_dict, f)


In [28]:
[author.split('*')[0].strip() for author in listOfAuthors]

['Peach, R. L.',
 'Landry, N. W.',
 'Cencetti, G.',
 'Gruber, D.',
 'Carletti, T.',
 'Robiglio, T.',
 'Tichit, L.',
 'Expert, P.',
 'Schwarze, A.',
 'Petit, J.',
 'Barrat, A.',
 'Vaccarino, F.',
 'Poetto, S.',
 'Lucas, M.',
 'Morris, A.',
 'Iacopini',
 'Battiston, F.',
 'Lucas',
 'Torres, L.',
 'Newman, J.',
 'Leit{\\~a}o, A.',
 'Iacopini, I.',
 'Habermann, B. H.',
 'Young, J.-G.',
 'Duncan, R.',
 'Hersh, T. A.',
 'Nurisso, M.',
 'Patania, A.',
 'Petri, G.',
 'Arnaudon, A.',
 'Zhang',
 'Stefanovska, A.',
 'Townsend-Teague, A.',
 'Bronstein, M.',
 'Latora, V.',
 'Gero, S.',
 'Fanelli, D.']